In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


# Data Download

In [ ]:
!gdown 1BIl9Ln24p8NS00rrRH0beDe_FkTlezfG

Downloading...
From: https://drive.google.com/uc?id=1BIl9Ln24p8NS00rrRH0beDe_FkTlezfG
To: /content/traffic_buddy_train+public_test.zip
100% 137k/137k [00:00<00:00, 113MB/s]


In [ ]:
!unzip /content/traffic_buddy_train+public_test.zip

Archive:  /content/traffic_buddy_train+public_test.zip
   creating: traffic_buddy_train+public_test/
  inflating: __MACOSX/._traffic_buddy_train+public_test  
  inflating: traffic_buddy_train+public_test/.DS_Store  
  inflating: __MACOSX/traffic_buddy_train+public_test/._.DS_Store  
   creating: traffic_buddy_train+public_test/public_test/
  inflating: __MACOSX/traffic_buddy_train+public_test/._public_test  
  inflating: traffic_buddy_train+public_test/readme.md  
  inflating: __MACOSX/traffic_buddy_train+public_test/._readme.md  
   creating: traffic_buddy_train+public_test/train/
  inflating: __MACOSX/traffic_buddy_train+public_test/._train  
  inflating: traffic_buddy_train+public_test/download_videos.py  
  inflating: __MACOSX/traffic_buddy_train+public_test/._download_videos.py  
  inflating: traffic_buddy_train+public_test/public_test/public_test.json  
  inflating: __MACOSX/traffic_buddy_train+public_test/public_test/._public_test.json  
  inflating: traffic_buddy_train+public_t

In [ ]:
%cd /content/traffic_buddy_train+public_test

/content/traffic_buddy_train+public_test


In [ ]:
!python download_videos.py


Found 182 unique videos to download from 405 records
Downloading: public_test/videos/2defe193_985_clip_002_0009_0019_N.mp4
2defe193_985_clip_002_0009_0019_N.mp4: 100% 24.9M/24.9M [00:00<00:00, 55.5MB/s]
Downloading: public_test/videos/49c10669_1000_clip_005_0031_0036_Y.mp4
49c10669_1000_clip_005_0031_0036_Y.mp4: 100% 15.0M/15.0M [00:00<00:00, 48.7MB/s]
Downloading: public_test/videos/fb16e5db_884_clip_003_0020_0025_Y.mp4
fb16e5db_884_clip_003_0020_0025_Y.mp4: 100% 3.72M/3.72M [00:00<00:00, 19.8MB/s]
Downloading: public_test/videos/92ca525a_1000_clip_003_0015_0023_Y.mp4
92ca525a_1000_clip_003_0015_0023_Y.mp4: 100% 20.3M/20.3M [00:00<00:00, 57.8MB/s]
Downloading: public_test/videos/93dfd907_792_clip_007_0042_0052_N.mp4
93dfd907_792_clip_007_0042_0052_N.mp4: 100% 28.4M/28.4M [00:00<00:00, 72.1MB/s]
Downloading: public_test/videos/0758e201_800_clip_014_0102_0109_Y.mp4
0758e201_800_clip_014_0102_0109_Y.mp4: 100% 10.0M/10.0M [00:00<00:00, 40.0MB/s]
Downloading: public_test/videos/238d6f44_80

# Qwen/Qwen3-VL-2B-Instruct

HuggingFace: https://huggingface.co/Qwen/Qwen3-VL-2B-Instruct




## Enviroment Setup

In [ ]:
!pip uninstall -y sentence-transformers  # Remove conflicting package
!pip install -qq git+https://github.com/huggingface/transformers  # Latest transformers cho Qwen3-VL
!pip install -qq torch torchvision accelerate pillow opencv-python
!pip install -qq flash-attn --no-build-isolation  # Flash Attention 2 cho tốc độ tối ưu

Found existing installation: sentence-transformers 5.1.1
Uninstalling sentence-transformers-5.1.1:
  Successfully uninstalled sentence-transformers-5.1.1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.0/502.0 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 120.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
VRAM: 79.3 GB


In [ ]:
# Kiểm tra Flash Attention 2 installation
try:
    import flash_attn
    print(f" Flash Attention 2 installed: version {flash_attn.__version__}")
    flash_attn_available = True
except ImportError:
    print("  Flash Attention 2 not installed - will use standard attention")
    print(" Standard attention is still very fast on A100!")
    flash_attn_available = False

 Flash Attention 2 installed: version 2.8.3


## Data Structure Check

In [ ]:
import os
import json

drive_path = "/content/traffic_buddy_train+public_test"

# Kiểm tra xem thư mục có tồn tại không
if os.path.exists(drive_path):
    print(f" Found TrafficBuddy dataset at: {drive_path}")

    # Kiểm tra train.json
    train_json_path = os.path.join(drive_path, "train/train.json")
    if os.path.exists(train_json_path):
        print(f" Found train.json at: {train_json_path}")
    else:
        print(f" train.json not found at: {train_json_path}")

    # Kiểm tra thư mục videos
    train_videos_path = os.path.join(drive_path, "train/videos")
    if os.path.exists(train_videos_path):
        video_count = len([f for f in os.listdir(train_videos_path) if f.endswith('.mp4')])
        print(f" Found {video_count} videos in: {train_videos_path}")
    else:
        print(f" Video folder not found at: {train_videos_path}")

else:
    print(f" TrafficBuddy dataset not found at: {drive_path}")
    print("Please update the 'drive_path' variable with correct path to your dataset")
    print(" Expected structure:")
    print("   /content/drive/MyDrive/traffic_buddy/")
    print("   ├── train/")
    print("   │   ├── train.json")
    print("   │   └── videos/")
    print("   │       ├── video1.mp4")
    print("   │       └── ...")
    print("   └── public_test/")
    print("       ├── public_test.json")
    print("       └── videos/")

 Found TrafficBuddy dataset at: /content/traffic_buddy_train+public_test
 Found train.json at: /content/traffic_buddy_train+public_test/train/train.json
 Found 532 videos in: /content/traffic_buddy_train+public_test/train/videos


## Load Model from HuggingFace

In [ ]:

from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
import torch

model_name = "Qwen/Qwen3-VL-2B-Instruct"


try:
    print("Attempting to load with Flash Attention 2...")
    model = Qwen3VLForConditionalGeneration.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2",
        device_map="auto",
    )
    print(" Successfully loaded with Flash Attention 2!")
except Exception as e:
    print(f" Flash Attention 2 failed: {e}")

    model = Qwen3VLForConditionalGeneration.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    print(" Successfully loaded with standard attention!")


processor = AutoProcessor.from_pretrained(model_name)

print("Qwen3-VL-2B-Instruct loaded successfully!")
print(f"Model memory usage: {torch.cuda.memory_allocated(0) / 1024**3:.1f} GB / {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Attempting to load with Flash Attention 2...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

Qwen3-VL-2B-Instruct loaded successfully!
Model memory usage: 4.0 GB / 79.3 GB


## Load Dataset

In [ ]:

train_json_path = os.path.join(drive_path, "train/train.json")

with open(train_json_path, 'r', encoding='utf-8') as f:
    train_data = json.load(f)['data']

print(f"Loaded {len(train_data)} training samples")

# videos available
available_samples = []
for sample in train_data:
    original_video_path = sample['video_path']
    drive_video_path = os.path.join(drive_path, original_video_path)

    if os.path.exists(drive_video_path):
        sample['video_path'] = drive_video_path
        available_samples.append(sample)

print(f"Found {len(available_samples)} samples with available videos in Google Drive")

# chọn số lượng videos để test zero-shot Model
test_samples = available_samples[:50]
print(f"Testing with {len(test_samples)} samples")

Loaded 1490 training samples
Found 1449 samples with available videos in Google Drive
Testing with 50 samples


## Video Processing

In [ ]:
import cv2
import numpy as np
from PIL import Image

def extract_frames_from_video(video_path, max_frames=12):
    """Extract more frames với better sampling"""
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps if fps > 0 else 0

    # print(f" Video info: {total_frames} frames, {fps:.1f} FPS, {duration:.1f}s")

    # Sample more densely for short videos
    if duration < 10:  # For short videos, sample every 0.5s
        frame_interval = max(1, int(fps * 0.5))
        frame_indices = list(range(0, total_frames, frame_interval))[:max_frames]
    else:
        frame_indices = np.linspace(0, total_frames-1, max_frames, dtype=int)

    frames = []
    for frame_idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(frame_rgb))

    cap.release()
    print(f" Extracted {len(frames)} frames")
    return frames

def format_question_with_choices(question, choices):
    """
    Format question with multiple choice options
    """
    formatted = f"Câu hỏi: {question}\n\nCác lựa chọn:\n"
    for choice in choices:
        formatted += f"{choice}\n"
    formatted += "\nHãy chọn đáp án đúng (chỉ trả lời A, B, C, hoặc D):"
    return formatted

## Predict

In [ ]:
def predict_answer(video_path, question, choices):
    """
    Predict answer using Qwen3-VL-2B model - Updated với official API
    """
    try:
        # Extract frames from video
        frames = extract_frames_from_video(video_path, max_frames=12)

        if not frames:
            return None, "Cannot extract frames from video"

        # Format question
        formatted_question = format_question_with_choices(question, choices)

        # Prepare messages theo official format
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": frames},  # Video frames
                    {"type": "text", "text": formatted_question},
                ],
            }
        ]

        # Sử dụng official inference method
        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt"
        )
        inputs = inputs.to(model.device)

        # Generate response
        generated_ids = model.generate(**inputs, max_new_tokens=128)
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]

        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )[0]

        return output_text.strip(), None

    except Exception as e:
        return None, str(e)

## Inference

In [ ]:
import time
from tqdm import tqdm

total_predictions = 0
correct_predictions = 0
results = []

print(f"Starting evaluation on {len(test_samples)} samples...")

for i, sample in enumerate(tqdm(test_samples)):
    video_path = sample['video_path']
    question = sample['question']
    choices = sample['choices']
    correct_answer = sample['answer']

    print(f"\n--- Sample {i+1}/{len(test_samples)} ---")
    print(f"Video: {video_path}")
    print(f"Question: {question}")
    print(f"Choices: {choices}")
    print(f"Correct answer: {correct_answer}")

    # Predict
    start_time = time.time()
    response, error = predict_answer(video_path, question, choices)
    inference_time = time.time() - start_time
    print("")

    print(f"Model response: {response}")
    print(f"Inference time: {inference_time:.2f}s")

    # Check đúng / sai
    is_correct = (response.strip().upper().startswith(correct_answer[0].upper()))
    if is_correct:
        correct_predictions += 1

    total_predictions += 1

    results.append({
        'sample_id': sample.get('id', f'sample_{i}'),
        'correct': correct_answer,
        'response': response,
        'inference_time': inference_time,
        'is_correct': is_correct
    })

# === Thống kê kết quả ===
accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\n✅ Evaluation completed: {correct_predictions}/{total_predictions} correct "
      f"({accuracy:.2f}% accuracy)")


Starting evaluation on 50 samples...


  0%|          | 0/50 [00:00<?, ?it/s]


--- Sample 1/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/2b840c67_386_clip_002_0008_0018_Y.mp4
Question: Nếu xe ô tô đang chạy ở làn ngoài cùng bên phải trong video này thì xe đó chỉ được phép rẽ phải?
Choices: ['A. Đúng', 'B. Sai']
Correct answer: B. Sai
 Extracted 12 frames


  2%|▏         | 1/50 [00:07<05:45,  7.06s/it]


Model response: B. Sai
Inference time: 7.06s

--- Sample 2/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/2b840c67_386_clip_002_0008_0018_Y.mp4
Question: Phần đường trong video cho phép các phương tiện đi theo hướng nào khi đến nút giao?
Choices: ['A. Đi thẳng', 'B. Đi thẳng và rẽ phải', 'C. Đi thẳng, rẽ trái và rẽ phải', 'D. Rẽ trái và rẽ phải']
Correct answer: C. Đi thẳng, rẽ trái và rẽ phải
 Extracted 12 frames


...


Model response: A
Inference time: 5.56s

--- Sample 49/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/23245211_035_clip_008_0051_0060_Y.mp4
Question: Xe đang đi trên làn hiện tại có được quay đầu không?
Choices: ['A. Có', 'B. Không']
Correct answer: B. Không
 Extracted 12 frames


 98%|█████████▊| 49/50 [04:38<00:05,  5.46s/it]


Model response: B. Không
Inference time: 5.58s

--- Sample 50/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/98543881_183_clip_001_0000_0007_Y.mp4
Question: Dựa vào video, xe ở làn nào (tính từ trái sang phải) được phép đi thẳng thẳng?
Choices: ['A. 1', 'B. 2', 'C. 3', 'D. Tất cả']
Correct answer: D. Tất cả
 Extracted 12 frames


100%|██████████| 50/50 [04:44<00:00,  5.69s/it]


Model response: A. 1
Inference time: 6.09s

✅ Evaluation completed: 30/50 correct (60.00% accuracy)


# Explore why the model choose this option

In [ ]:
def predict_answer_with_reason(video_path, question, choices):
    """
    Predict answer using Qwen3-VL-2B model (official API) + reasoning output
    """
    try:
        frames = extract_frames_from_video(video_path, max_frames=12)
        if not frames:
            return None, "Cannot extract frames from video"

        formatted_question = (
            "Bạn là trợ lý giao thông Việt Nam. "
            "Dựa vào video dashcam, hãy chọn đúng 1 đáp án "
            "và giải thích ngắn gọn vì sao chọn đáp án đó.\n\n"
            f"Câu hỏi: {question}\n"
            "Các lựa chọn:\n" + "\n".join(choices) +
            "\n\nTrả lời theo định dạng:\nAnswer: <A/B/C/D>\nReason: <ngắn gọn>"
        )

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": frames},
                    {"type": "text", "text": formatted_question},
                ],
            }
        ]

        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device)

        generated_ids = model.generate(**inputs, max_new_tokens=128)
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )[0].strip()

        import re
        answer_match = re.search(r"Answer:\s*([ABCD])", output_text, re.I)
        reason_match = re.search(r"Reason:\s*(.*)", output_text, re.I)

        answer = answer_match.group(1).upper() if answer_match else "?"
        reason = reason_match.group(1).strip() if reason_match else "(Không rõ lý do)"

        return f"{answer} — {reason}", None

    except Exception as e:
        return None, str(e)


In [ ]:
import time
from tqdm import tqdm

total_predictions = 0
correct_predictions = 0
results = []

print(f"Starting evaluation on {len(test_samples)} samples...")

for i, sample in enumerate(tqdm(test_samples)):
    video_path = sample['video_path']
    question = sample['question']
    choices = sample['choices']
    correct_answer = sample['answer']

    print(f"\n--- Sample {i+1}/{len(test_samples)} ---")
    print(f"Video: {video_path}")
    print(f"Question: {question}")
    print(f"Choices: {choices}")
    print(f"Correct answer: {correct_answer}")

    # Predict
    start_time = time.time()
    response, error = predict_answer_with_reason(video_path, question, choices)
    inference_time = time.time() - start_time
    print("")

    print(f"Model response: {response}")
    print(f"Inference time: {inference_time:.2f}s")

    # Check đúng / sai
    is_correct = (response.strip().upper().startswith(correct_answer[0].upper()))
    if is_correct:
        correct_predictions += 1

    total_predictions += 1

    results.append({
        'sample_id': sample.get('id', f'sample_{i}'),
        'correct': correct_answer,
        'response': response,
        'inference_time': inference_time,
        'is_correct': is_correct
    })

# === Thống kê kết quả ===
accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
print(f"\n✅ Evaluation completed: {correct_predictions}/{total_predictions} correct "
      f"({accuracy:.2f}% accuracy)")

Starting evaluation on 50 samples...


  0%|          | 0/50 [00:00<?, ?it/s]


--- Sample 1/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/2b840c67_386_clip_002_0008_0018_Y.mp4
Question: Nếu xe ô tô đang chạy ở làn ngoài cùng bên phải trong video này thì xe đó chỉ được phép rẽ phải?
Choices: ['A. Đúng', 'B. Sai']
Correct answer: B. Sai
 Extracted 12 frames


  2%|▏         | 1/50 [00:08<06:53,  8.43s/it]


Model response: B — Xe đang chạy ở làn ngoài cùng bên phải, nên không thể rẽ phải vì không có đủ không gian.
Inference time: 8.43s

--- Sample 2/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/2b840c67_386_clip_002_0008_0018_Y.mp4
Question: Phần đường trong video cho phép các phương tiện đi theo hướng nào khi đến nút giao?
Choices: ['A. Đi thẳng', 'B. Đi thẳng và rẽ phải', 'C. Đi thẳng, rẽ trái và rẽ phải', 'D. Rẽ trái và rẽ phải']
Correct answer: C. Đi thẳng, rẽ trái và rẽ phải
 Extracted 12 frames


...


Model response: A — Trong video có thể thấy rõ vạch kẻ đường trên mặt đường, và các phương tiện đang di chuyển theo hướng vạch kẻ đường. Không có biển chỉ dẫn cụ thể như cầu vượt hoặc đường dành cho xe khách.
Inference time: 8.16s

--- Sample 49/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/23245211_035_clip_008_0051_0060_Y.mp4
Question: Xe đang đi trên làn hiện tại có được quay đầu không?
Choices: ['A. Có', 'B. Không']
Correct answer: B. Không
 Extracted 12 frames


 98%|█████████▊| 49/50 [06:14<00:07,  7.65s/it]


Model response: B — Xe đang ở trong làn đường đang chờ đèn đỏ, không có dấu hiệu nào cho thấy nó đang quay đầu.
Inference time: 7.08s

--- Sample 50/50 ---
Video: /content/traffic_buddy_train+public_test/train/videos/98543881_183_clip_001_0000_0007_Y.mp4
Question: Dựa vào video, xe ở làn nào (tính từ trái sang phải) được phép đi thẳng thẳng?
Choices: ['A. 1', 'B. 2', 'C. 3', 'D. Tất cả']
Correct answer: D. Tất cả
 Extracted 12 frames


100%|██████████| 50/50 [06:22<00:00,  7.66s/it]


Model response: A — <Xe ở làn 1, được phép đi thẳng. Trên đường có vạch kẻ chỉ định làn đi thẳng, và đèn tín hiệu cho phép đi thẳng.>
Inference time: 8.01s

✅ Evaluation completed: 30/50 correct (60.00% accuracy)
